# Saving joined data

In [ ]:
import pandas as pd, numpy as np
from tqdm.auto import tqdm
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Three datasets
url_members = 'https://dataspace.princeton.edu/bitstream/88435/dsp01dv13zx35z/2/SCoData_members_v1.2_2022-01.csv'
url_books = 'https://dataspace.princeton.edu/bitstream/88435/dsp01jm214s28p/2/SCoData_books_v1.2_2022-01.csv'
url_events = 'https://dataspace.princeton.edu/bitstream/88435/dsp019306t2441/2/SCoData_events_v1.2_2022-01.csv'

# load in pandas
df_members = pd.read_csv(url_members).fillna('')
df_books = pd.read_csv(url_books).fillna('')
df_events = pd.read_csv(url_events).fillna('')

In [ ]:
df_members.columns

Index(['uri', 'name', 'sort_name', 'title', 'gender', 'is_organization',
       'has_card', 'birth_year', 'death_year', 'membership_years', 'viaf_url',
       'wikipedia_url', 'nationalities', 'addresses', 'postal_codes',
       'arrondissements', 'coordinates', 'notes', 'updated'],
      dtype='object')

In [ ]:
df_books.columns

Index(['uri', 'title', 'author', 'editor', 'translator', 'introduction',
       'illustrator', 'photographer', 'year', 'format', 'uncertain',
       'ebook_url', 'volumes_issues', 'notes', 'event_count', 'borrow_count',
       'purchase_count', 'circulation_years', 'updated'],
      dtype='object')

In [ ]:
df_events.columns

Index(['event_type', 'start_date', 'end_date', 'member_uris', 'member_names',
       'member_sort_names', 'subscription_price_paid', 'subscription_deposit',
       'subscription_duration', 'subscription_duration_days',
       'subscription_volumes', 'subscription_category',
       'subscription_purchase_date', 'reimbursement_refund', 'borrow_status',
       'borrow_duration_days', 'purchase_price', 'currency', 'item_uri',
       'item_title', 'item_volume', 'item_authors', 'item_year', 'item_notes',
       'source_type', 'source_citation', 'source_manifest', 'source_image'],
      dtype='object')

In [ ]:
def getrowinfo(row, i):
    odx={}
    for k,v in dict(row).items():
        if ';' in str(v):
            vs = str(v).split(';')
            try:
                v=vs[i]
            except IndexError:
                print('!!',i,vs)
        odx[k]=v
    return odx

In [ ]:
df_events2 = pd.DataFrame(
    {**getrowinfo(row,mi), 'member_uri':muri}
    for i,row in tqdm(df_events.iterrows(), total=len(df_events))
    for mi,muri in enumerate(row.member_uris.split(';'))
)

len(df_events), len(df_events2)

  0%|          | 0/35523 [00:00<?, ?it/s]

(35523, 36080)

In [ ]:
# make sure no semicolons surviving
for col in df_events2.columns: assert True not in set(df_events2[col].str.contains(';'))

In [ ]:
# join all datasets -- events link books and people
df = df_events2.merge(
    df_members, 
    left_on='member_uris',
    right_on='uri',
    suffixes=('_event','_member')
).merge(
    df_books,
    left_on='item_uri',
    right_on='uri',
    suffixes=('_event','_book')
)

In [ ]:
df['member_uri']=df['member_uris']
df['book_uri']=df['item_uri']
df['event_uri']=list(df.index)
cols=['member_uri','book_uri','event_uri']
df_smpl=df.loc[df[cols].dropna().index][cols + [c for c in df if c not in cols]]
df_smpl

,member_uri,book_uri,event_uri,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,...,format,uncertain,ebook_url,volumes_issues,notes_book,event_count,borrow_count,purchase_count,circulation_years,updated_book
0,https://shakespeareandco.princeton.edu/members...,https://shakespeareandco.princeton.edu/books/b...,0,Generic,1920,,https://shakespeareandco.princeton.edu/members...,Raymonde Linossier,"Linossier, Raymonde",,...,Book,False,https://archive.org/details/pigsispigs00unkngo...,,,2,1,0,1920;1919,2020-06-29T02:07:28+00:00
1,https://shakespeareandco.princeton.edu/members...,https://shakespeareandco.princeton.edu/books/b...,1,Borrow,1919-11-24,1919-12-02,https://shakespeareandco.princeton.edu/members...,Claude Cahun / Mlle Lucie Schwob,"Cahun, Claude",,...,Book,False,https://archive.org/details/pigsispigs00unkngo...,,,2,1,0,1920;1919,2020-06-29T02:07:28+00:00
2,https://shakespeareandco.princeton.edu/members...,https://shakespeareandco.princeton.edu/books/n...,2,Borrow,1920-12,,https://shakespeareandco.princeton.edu/members...,Raymonde Linossier,"Linossier, Raymonde",,...,Book,False,https://archive.org/details/shanghaiedstoryo00...,,,9,9,0,1920;1922;1923;1927;1941;1944;1947,2021-11-06T12:33:22.945119+00:00
3,https://shakespeareandco.princeton.edu/members...,https://shakespeareandco.princeton.edu/books/n...,3,Borrow,1944-09-19,1944-09-21,https://shakespeareandco.princeton.edu/members...,Monique de Vigan,"de Vigan, Monique",,...,Book,False,https://archive.org/details/shanghaiedstoryo00...,,,9,9,0,1920;1922;1923;1927;1941;1944;1947,2021-11-06T12:33:22.945119+00:00
4,https://shakespeareandco.princeton.edu/members...,https://shakespeareandco.princeton.edu/books/n...,4,Borrow,1947-06-02,1947-09-07,https://shakespeareandco.princeton.edu/members...,Monique de Vigan,"de Vigan, Monique",,...,Book,False,https://archive.org/details/shanghaiedstoryo00...,,,9,9,0,1920;1922;1923;1927;1941;1944;1947,2021-11-06T12:33:22.945119+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22821,https://shakespeareandco.princeton.edu/members...,https://shakespeareandco.princeton.edu/books/b...,22821,Gift,1962-06-28,,https://shakespeareandco.princeton.edu/members...,Jean-Dominique Rey,"Rey, Jean-Dominique",,...,Book,False,,,,1,0,0,1962,2020-07-05T03:26:49+00:00
22822,https://shakespeareandco.princeton.edu/members...,https://shakespeareandco.princeton.edu/books/a...,22822,Borrow,1962-06-28,,https://shakespeareandco.princeton.edu/members...,Jean-Dominique Rey,"Rey, Jean-Dominique",,...,Book,False,,,A catalog for an exhibition at the Center Cult...,1,1,0,1962,2020-07-19T20:08:16+00:00
22823,https://shakespeareandco.princeton.edu/members...,https://shakespeareandco.princeton.edu/books/s...,22823,Purchase,,,https://shakespeareandco.princeton.edu/members...,Mme le Moal,"Moal, Mme le",,...,Book,False,,,French translation of *Twelfth Night.*,1,0,1,,2020-07-19T15:38:29+00:00
22824,https://shakespeareandco.princeton.edu/members...,https://shakespeareandco.princeton.edu/books/m...,22824,Purchase,,,https://shakespeareandco.princeton.edu/members...,Mme le Moal,"Moal, Mme le",,...,,True,,,Unidentified. By or about Marie Stopes.,1,0,1,,2021-11-21T02:28:25.258248+00:00


In [ ]:
### SAVE TO GOOGLE DRIVE
sheet='GEOTASTE_LINKED_DATA_20230202'

from google.colab import auth
import gspread
from google.auth import default
from gspread import SpreadsheetNotFound
from gspread_dataframe import set_with_dataframe

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

try:
    # Open our new sheet and add some data.
    worksheet = gc.open(sheet).sheet1
except SpreadsheetNotFound:
    sh = gc.create(sheet)
    worksheet = gc.open(sheet).sheet1



In [ ]:
### ACTUALLY SAVE
# set_with_dataframe(worksheet, df_smpl, include_index=False, resize=True)